In [51]:
import joblib
import pandas as pd
import os
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline

def get_final_estimator(model):
    """
    Se for Pipeline, retorna o estimador final (step final).
    Caso contrário, retorna o próprio objeto.
    """
    if isinstance(model, Pipeline):
        # pipeline[-1] nem sempre funciona; vamos pegar named_steps
        try:
            # sklearn Pipeline tem attribute named_steps
            last_step = list(model.named_steps.items())[-1][1]
            return last_step
        except Exception:
            # fallback: acessar estimators_ ou steps
            try:
                return model.steps[-1][1]
            except Exception:
                return model
    return model

def detect_algorithm(model):
    """
    Retorna o nome amigável do algoritmo (classe) usado.
    """
    final = get_final_estimator(model)
    cls_name = final.__class__.__name__
    module = final.__class__.__module__
    return f"{cls_name} (module: {module})"

def try_feature_names_from_model(model):
    """
    Tenta recuperar feature names esperadas a partir do objeto salvo.
    Retorna lista de colunas ou None.
    Procura em:
     - model.feature_names_in_
     - pipeline.feature_names_in_
     - estimador final feature_names_in_
    """
    # direto no objeto
    for obj in (model, get_final_estimator(model)):
        if hasattr(obj, "feature_names_in_"):
            cols = list(getattr(obj, "feature_names_in_"))
            return cols

    # tentar atributo comum salvo manualmente (feature_names, feature_columns)
    for attr in ("feature_names", "feature_columns", "feature_cols", "columns"):
        if hasattr(model, attr):
            cols = getattr(model, attr)
            try:
                return list(cols)
            except Exception:
                pass
    # procurar arquivo salvo localmente
    for fname in ("feature_cols.pkl", "feature_columns.pkl", "feature_cols.json", "X_columns.pkl", "feature_cols.npy"):
        if os.path.exists(fname):
            try:
                if fname.endswith(".pkl"):
                    return list(joblib.load(fname))
                elif fname.endswith(".json"):
                    import json
                    with open(fname, "r", encoding="utf-8") as f:
                        return json.load(f)
                elif fname.endswith(".npy"):
                    return list(np.load(fname, allow_pickle=True))
            except Exception:
                pass
    return None

def align_input_to_expected(df_raw, expected_cols):
    """
    Converte df_raw para dummies/categoricas e reindexa para expected_cols,
    preenchendo colunas faltantes com 0.
    """
    # criar dummies para todas as colunas categóricas/objeto presentes
    dummies = pd.get_dummies(df_raw, drop_first=False)
    # reindex na ordem correta
    X = dummies.reindex(columns=expected_cols, fill_value=0)
    return X

def predict_with_model(model, df_raw):
    """
    Fluxo principal:
     - detecta algoritmo
     - tenta usar o pipeline diretamente
     - se houver erro de nomes de feature, tenta alinhar automaticamente
    Retorna (predicoes, info_dict)
    """
    info = {}
    info["detected_algorithm"] = detect_algorithm(model)
    info["is_pipeline"] = isinstance(model, Pipeline)
    info["used_columns"] = None
    info["note"] = None

    # 1) Se for Pipeline, primeiro tentamos predict direto com o DataFrame cru
    try:
        preds = model.predict(df_raw)
        info["note"] = "Predição executada com sucesso diretamente (pipeline aceita DataFrame cru)."
        # tentar inferir as colunas usadas (se pipeline tiver feature_names_in_)
        info["used_columns"] = try_feature_names_from_model(model) or list(df_raw.columns)
        return preds, info
    except Exception as e_pipeline:
        # se não for pipeline ou falhou com erro de nomes, anotamos
        info["pipeline_predict_error"] = str(e_pipeline)

    # 2) tentar detectar as feature names que foram usadas no fit
    expected_cols = try_feature_names_from_model(model)
    info["expected_columns_detected"] = expected_cols is not None

    if expected_cols is None:
        # não foi possível detectar automaticamente
        raise ValueError(
            "Não foi possível detectar automaticamente as colunas que o modelo espera.\n"
            "Opções:\n"
            " - Se você salvou as colunas (feature_cols) durante o treinamento, salve-as em 'feature_cols.pkl' e rode novamente.\n"
            " - Se o modelo foi salvo como Pipeline (pré-processador + estimator) carregue esse pipeline inteiro e use pipeline.predict(df_cru).\n"
            " - Se quiser, forneça manualmente a lista expected_cols e eu ajudo a reindexar.\n"
        )

    # 3) alinhar os dados mocados para as colunas esperadas
    X_aligned = align_input_to_expected(df_raw, expected_cols)
    info["used_columns"] = expected_cols

    # 4) prever com o modelo (convertendo para numpy se necessário)
    try:
        preds = model.predict(X_aligned)
        info["note"] = "Predição executada após alinhamento de colunas (one-hot / reindex)."
        return preds, info
    except Exception as e_final:
        # último recurso: tentar com o estimador final (se pipeline) usando numpy array
        final_est = get_final_estimator(model)
        try:
            preds = final_est.predict(X_aligned.values)
            info["note"] = "Predição executada com o estimador final usando numpy array (valide ordem de colunas manualmente)."
            return preds, info
        except Exception as e_final2:
            raise RuntimeError(
                "Falha ao executar predict mesmo após alinhamento.\n"
                f"Erro pipeline_predict: {info.get('pipeline_predict_error')}\n"
                f"Erro final_estimator_predict: {e_final}\n"
                f"Erro final_estimator_predict_numpy: {e_final2}\n"
            )

# ----------------------------
# Exemplo de uso
# ----------------------------
if __name__ == "__main__":
    # Carregar modelo (altere o caminho se necessário)
    modelo = joblib.load("mymodel.pkl")

    # Mostrar qual algoritmo está sendo usado
    print("Algoritmo detectado:", detect_algorithm(modelo))

    # Criar dados mocados (use as colunas originais; o script tentará converter)
    dados_mocados = pd.DataFrame([
        {
            "idade": 35,
            "sexo": "M",
            "posse_de_veiculo": 1,
            "posse_de_imovel": 0,
            "qtd_filhos": 2,
            "tipo_renda": "Assalariado",
            "educacao": "Higher education",       # use valores iguais aos do treino quando possível
            "estado_civil": "Solteiro",
            "tipo_residencia": "Aluguel",
            "renda": 4500
        },
        {
            "idade": 22,
            "sexo": "F",
            "posse_de_veiculo": 0,
            "posse_de_imovel": 0,
            "qtd_filhos": 0,
            "tipo_renda": "Bolsista",
            "educacao": "Secondary / secondary special",
            "estado_civil": "Casado",
            "tipo_residencia": "Com os pais",
            "renda": 1800
        }
    ])

    # Rodar predição via função segura
    preds, info = predict_with_model(modelo, dados_mocados)

    print("\nInfo:")
    for k, v in info.items():
        print(f"- {k}: {v}")

    print("\nPredições:", preds)


Algoritmo detectado: DecisionTreeClassifier (module: sklearn.tree._classes)

Info:
- detected_algorithm: DecisionTreeClassifier (module: sklearn.tree._classes)
- is_pipeline: False
- used_columns: ['qtd_filhos', 'idade', 'tempo_emprego', 'possui_celular', 'possui_fone_comercial', 'possui_fone', 'possui_email', 'qt_pessoas_residencia', 'sexo_M', 'posse_de_veiculo_Y', 'posse_de_imovel_Y', 'tipo_renda_Commercial associate', 'tipo_renda_Pensioner', 'tipo_renda_State servant', 'tipo_renda_Student', 'tipo_renda_Working', 'educacao_Academic degree', 'educacao_Higher education', 'educacao_Incomplete higher', 'educacao_Lower secondary', 'educacao_Secondary / secondary special', 'estado_civil_Civil marriage', 'estado_civil_Married', 'estado_civil_Separated', 'estado_civil_Single / not married', 'estado_civil_Widow', 'tipo_residencia_Co-op apartment', 'tipo_residencia_House / apartment', 'tipo_residencia_Municipal apartment', 'tipo_residencia_Office apartment', 'tipo_residencia_Rented apartment',